# [0] Imports

In [203]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets, Dates

In [204]:
LOW_JOBS = 19
HIGH_JOBS = 19;

In [205]:
times_pp_start = []
times_pp_end = []

times_mb_start = []
times_mb_end = []

times_op_start = []
times_op_end = []

ANSWERS = [] # record the actual routes
OBJECTIVES = [] # record the objective
;

In [206]:
for JOBS in LOW_JOBS:HIGH_JOBS
    # loop begins
    
    # SETUP: doesn't count in our time
    n_jobs = JOBS
    n_vehicles_jobs = 10
    n_vehicles_coverage = 10
    T = 2 * n_jobs
    min_duration = 2
    max_duration = 6
    speed = 1000/6
    coverage_distance = 50
    size = 500
    step = 50;
    
    Random.seed!(1234)

    function create_cluster_sizes(jobs_total)
        Random.seed!(1234)
        jobs_created = 0
        cluster = []
        min_jobs_allowed = n_jobs >= 15 ? 3 : 2
        target_clusters = 5
        while jobs_created != jobs_total
            num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
            jobs_created += num_to_add
            push!(cluster, num_to_add)
        end

        return cluster
    end;
    
    time_windows = []
    locations = rand(Uniform(0,size), 1, 2)
    work_load = []

    cluster = create_cluster_sizes(n_jobs);
    
    function create_time_windows_and_work_load(cluster_sizes, locations)
        Random.seed!(1234)
        locations = rand(Uniform(0,size), 1, 2)
        for size_c in cluster
            first = rand(Uniform(0,size), 1, 2)
            locations = vcat(locations, first)

            job_begins = rand(2:10)
            job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
            push!(work_load, time_work)

            for neighbour in 1:(size_c-1)
                new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
                new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
                new = hcat(new_x, new_y)
                locations = vcat(locations, new)

                job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
                job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
                push!(time_windows, [job_begins, job_finish])

                time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
                push!(work_load, time_work)
            end
        end

        return [time_windows, work_load, locations]
    end;
    
    time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
    distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];
    
    # THIS IS WHERE TIMING OF PRE-PROCESSING BEGINS
    push!(times_pp_start, datetime2unix(now()))
    function create_node_network(n_jobs, T)
        nodes = []
        for i in 1:n_jobs
            for t in 2:(T-1)
                push!(nodes, [i, t])
            end
        end
        return nodes
    end;
    
    nodes = create_node_network(n_jobs, T)
    N = length(nodes);
    
    function create_arc_system(n_jobs, distances, speed, time_windows, nodes)
        #=
        Create a full arc system which has the format
        [ [initial_i, initial_t], [end_i, end_t], distance]

        remember: distance is like our cost, the first two elements represent the movement
        =#

        arcs = []

        #leaving the depot
        for n in nodes
            dist = distances[1, n[1] + 1]
            max_t = 0
            for t in 1:n[2]
                if n[2] > (t + dist/speed)
                    max_t = t
                end
            end
            push!(arcs, [[0, max_t], n, dist])
        end

        #between two jobs
        for n1 in nodes
            already = []
            for n2 in nodes
                dist = distances[n1[1]+1, n2[1]+1]

                #=
                only add arc from node n1 = [i1, t1] to n2 = [i2, t2] if:
                1) it is possible to get from n1 to n2 within the given timeframe (time = dist / speed)
                2) the nodes do not have the same job location - this is covered in elseif
                3) n2 is not already covered in our node network - it won't be a superfluous addition
                4) n2's time is worth going to: we have enough time left to get there to do the job

                condition 4 example: If current time is 15, time window is [13, 20], and the work load 
                (time required) is 6 hours, then don't bother going to the job - it's a waste of time
                =#
                if (n2[2] > n1[2] + dist/speed) & 
                    (n2[1] != n1[1]) & 
                    (!(n2[1] in  already)) & 
                    (n2[2] <= (time_windows[n2[1]][2] - work_load[n2[1]]))
                    push!(arcs, [n1, n2, dist])
                    push!(already, n2[1]) #already visited so we don't do it again: algorithmic efficiency

                elseif (n2[2] == n1[2] + 1) & (n2[1] == n1[1])
                    push!(arcs, [n1, n2, dist])

                end
            end
        end

        #Back to the depot
        for n in nodes
            dist = distances[1, n[1] + 1]
            min_t = T+1
            for t in T:-1:1
                if t > (n[2]+dist/speed)
                    min_t = t
                end
            end
            push!(arcs, [n, [n_jobs+1, min_t], dist])
        end

        #Now, add intra-depot arcs to minimize drivers necessary
        push!(arcs, [[0,1], [n_jobs+1, 2], 0])

        return arcs
    end;
    
    arcs = create_arc_system(n_jobs, distances, speed, time_windows, nodes)
    A = length(arcs);
    
    function find_node(i,t)
        for node in 1:length(nodes)
            if (nodes[node][1] == i) & (nodes[node][2] == t) 
                return node
            end
        end
        return "Warning, this node does not exist"
    end;
    
    function find_arc(node, t)
        for arc in 1:length(arcs)
            if (arcs[arc][1][1] == node) & (arcs[arc][2][1] == node) & ((arcs[arc][1][2] == t)) & ((arcs[arc][2][2] == t+1))
                return arc
            end
        end
        return (node, t), "Warning, this arc does not exist"
    end;
    # THIS IS WHERE PRE-PROCESSING ENDS
    push!(times_pp_end, datetime2unix(now()))
    
    model = Model(Gurobi.Optimizer);
    set_optimizer_attribute(model, "OutputFlag", 0);
    
    # THIS IS WHERE MODEL-BUILDING STARTS
    push!(times_mb_start, datetime2unix(now()))
    
    @variable(model, z[1:n_vehicles_jobs, 1:A], Bin)
    @variable(model, y_s[1:n_jobs, 1:T], Bin)
    @variable(model, y_e[1:n_jobs, 1:T], Bin);
    
    @objective(model, Min,
        sum(sum(z[k,a]*(arcs[a][3]+1) for a in 1:A) for k in 1:n_vehicles_jobs));
    
    @constraint(model, 
        depot_init_jobs[k in 1:n_vehicles_jobs], 
        sum(z[k,a] for a in 1:A if arcs[a][1][1] == 0) == 1);
    @constraint(model, 
        depot_end_jobs[k in 1:n_vehicles_jobs], 
        sum(z[k,a] for a in 1:A if arcs[a][2][1] == n_jobs+1) == 1);
    @constraint(model, 
        flow_jobs[k in 1:n_vehicles_jobs, n in 1:N], 
        sum(z[k,a] for a in 1:A if arcs[a][2] == nodes[n]) == 
        sum(z[k,a] for a in 1:A if arcs[a][1] == nodes[n]));
    @constraint(model, 
        unique[n in 1:n_jobs], 
        sum(sum(z[k,a] for a in 1:A 
                    if (arcs[a][2][1] == n) & (arcs[a][1][1] != n)) 
            for k in 1:n_vehicles_jobs) == 1);
    @constraint(model, start[i in 1:n_jobs, t in 1:(T-1)], y_s[i,t] <= y_s[i,t+1])
    @constraint(model, ends[i in 1:n_jobs, t in 1:(T-1)], y_e[i,t] <= y_e[i,t+1])
    @constraint(model, 
        work[i in 1:n_jobs, t in 2:(T-2)], 
        y_s[i,t] - y_e[i,t] <= sum(z[k,find_arc(i, t)] for k in 1:n_vehicles_jobs))
    @constraint(model, window_start[i in 1:n_jobs, t in 1:(time_windows[i][1]-1)], y_s[i,t] <= 0)
    @constraint(model, window_ends[i in 1:n_jobs, t in time_windows[i][2]:T], y_e[i,t] >= 1)
    @constraint(model, duration[i in 1:n_jobs], sum(y_s[i,t] - y_e[i,t] for t in 1:T) >= work_load[i]);
    
    # THIS IS WHERE MODEL-BUILDING ENDS
    push!(times_mb_end, datetime2unix(now()))
    
    # THIS IS WHERE OPT STARTS
    push!(times_op_start, datetime2unix(now()))
    optimize!(model);
    #THIS IS WHERE OPT ENDS
    push!(times_op_end, datetime2unix(now()))
    
    push!(OBJECTIVES, objective_value(model))
    
    # SEE SOLUTION - not included in time computations.
    z_ka_v = value.(z); # this is a n_jobs x A matrix.
    
    nab_array = []
    for vehicle_num in 1:n_vehicles_jobs
        list_of_1s = []
        for arc_index in 1:A
            if z_ka_v[vehicle_num, arc_index] > 0.99
                push!(list_of_1s, arcs[arc_index])
            end
        end

        push!(nab_array, list_of_1s)
    end
    
    for list in nab_array
        sort!(list, lt = (x, y) -> isless(x[1][2], y[1][2]))
    end
    
    visited_jobs = []
    for vehicle_num in 1:n_vehicles_jobs
        arcs_used = nab_array[vehicle_num]
        vehicle_visited_jobs = []
        for i in 1:length(arcs_used)
            first_elt = arcs_used[i][1][1]
            if ~(first_elt in vehicle_visited_jobs)
                push!(vehicle_visited_jobs, first_elt)
            end
        end

        push!(vehicle_visited_jobs, n_jobs+1)
        push!(visited_jobs, vehicle_visited_jobs)
    end
    
    answer_list = []
    for job_list in visited_jobs
        if length(job_list) > 2
            push!(answer_list, job_list)
        end
    end
    push!(ANSWERS, answer_list)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [207]:
pp_arr = times_pp_end .- times_pp_start
mb_arr = times_mb_end .- times_mb_start
op_arr = times_op_end .- times_op_start
;

In [208]:
pp_arr

1-element Vector{Float64}:
 0.40599989891052246

In [209]:
mb_arr

1-element Vector{Float64}:
 16.38100004196167

In [210]:
op_arr

1-element Vector{Float64}:
 5.177999973297119

In [211]:
total_arr = pp_arr .+ mb_arr .+ op_arr

1-element Vector{Float64}:
 21.96499991416931

In [212]:
ANSWERS

1-element Vector{Any}:
 Any[Any[0, 19, 7, 8, 9, 20], Any[0, 1, 2, 3, 20], Any[0, 4, 5, 6, 20], Any[0, 16, 17, 18, 20], Any[0, 13, 14, 15, 20], Any[0, 10, 11, 12, 20]]

In [213]:
OBJECTIVES

1-element Vector{Any}:
 3154.732890636053